# Spatial Comps for Vacant Land Valuation in Baltimore — **EDA Notebook**
This notebook performs **exploratory data analysis (EDA)** for the UMBC DATA606 capstone. We validate joins between Real Property and Parcel datasets, compute **price per square foot (USD/ft²)** for **vacant land** sales, and visualize key drivers aligned with the proposal: time trends, lot-size effects, and location proxies (Ward/Neighborhood).

### Mount Google Drive
**What this does**: Connect Colab to Google Drive so files can be read and written.
**Key outputs**: Drive mounted at /content/drive.

In [ ]:
from google.colab import drive
drive.flush_and_unmount()        # cleanly unmount if mounted
!fusermount -u /content/drive || true
!rm -rf /content/drive           # remove leftover local folder (not your real Drive)
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
fusermount: failed to unmount /content/drive: No such file or directory
Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### General setup

Reading the dataset

In [ ]:
DRIVE_FOLDER = "/content/drive/MyDrive/Capstone"

real_df  = f"{DRIVE_FOLDER}/Real_Property_Information.csv"
parcel_df = f"{DRIVE_FOLDER}/Parcel.geojson"

out_dir = f"{DRIVE_FOLDER}/processed"
!mkdir -p "$out_dir"

print("Using paths:")
print("  REAL_PATH_CSV  :", real_df)
print("  PARCEL_PATH_GEO:", parcel_df)
print("  OUT_DIR        :", out_dir)


Using paths:
  REAL_PATH_CSV  : /content/drive/MyDrive/Capstone/Real_Property_Information.csv
  PARCEL_PATH_GEO: /content/drive/MyDrive/Capstone/Parcel.geojson
  OUT_DIR        : /content/drive/MyDrive/Capstone/processed


### **Real Property Dataset**

In [ ]:
import pandas as pd
real = pd.read_csv(real_df, low_memory=False)
print("Shape:", real.shape)
print("Columns (first 20):", list(real.columns)[:20])
real.head(3)

Shape: (238360, 79)
Columns (first 20): ['OBJECTID', 'PIN', 'PINRELATE', 'BLOCKLOT', 'BLOCK', 'LOT', 'WARD', 'SECTION', 'ASSESSOR', 'TAXBASE', 'BFCVLAND', 'BFCVIMPR', 'LANDEXMP', 'IMPREXMP', 'CITYCRED', 'STATCRED', 'CCREDAMT', 'SCREDAMT', 'PERMHOME', 'ASSESGRP']


,OBJECTID,PIN,PINRELATE,BLOCKLOT,BLOCK,LOT,WARD,SECTION,ASSESSOR,TAXBASE,...,YEAR_BUILD,STRUCTAREA,LDATE,OWNMDE,GRNDRENT,SUBTYPE_GEODB,SDATLINK,BLOCKPLAT,MAILTOADD,VACIND
0,1,1001,1001,0001 001,1,1,15,370,329,172000,...,1920,4344.0,8312025,F,0.0,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"1639 MORELAND AVE, 21216",NaN
1,2,1002,1002,0001 002,1,2,15,370,329,49500,...,0,0.0,8312025,F,0.0,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"1639 MORELAND AVE, 21216",NaN
2,3,1003,1003,0001 003,1,3,15,370,377,9000,...,1915,0.0,8312025,,0.0,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"HILTON PLAZA STE 201, 21208",Y


Basic checks like nulls, duplicates etc.

In [ ]:

key_cols = ['PIN','BLOCKLOT','SALEPRIC','SALEDATE','NO_IMPRV','STRUCTAREA','CURRIMPR','BFCVIMPR','WARD','SECTION','NEIGHBOR']
for c in key_cols:
    if c not in real.columns:
        real[c] = None

nulls = real[key_cols].isna().sum().sort_values(ascending=False)
print("Null counts (key columns):\n", nulls, "\n")



dup_all = real.duplicated().sum()
print("Duplicate full rows:", dup_all)


subset_cols = ['PIN','SALEDATE','SALEPRIC']
subset_cols = [c for c in subset_cols if c in real.columns]
dup_subset = real.duplicated(subset=subset_cols).sum() if subset_cols else 0
print(f"Duplicate rows on {subset_cols}:", dup_subset)


print("\nNO_IMPRV top values:\n", real['NO_IMPRV'].astype(str).str.strip().str.upper().value_counts(dropna=False).head(5))
print("\nWARD top values:\n", real['WARD'].astype(str).str.strip().str.upper().value_counts(dropna=False).head(5))
print("\nSECTION top values:\n", real['SECTION'].astype(str).str.strip().str.upper().value_counts(dropna=False).head(5))


Null counts (key columns):
 NO_IMPRV      217437
STRUCTAREA      1671
PIN                0
SALEPRIC           0
BLOCKLOT           0
SALEDATE           0
CURRIMPR           0
BFCVIMPR           0
WARD               0
SECTION            0
NEIGHBOR           0
dtype: int64 

Duplicate full rows: 0
Duplicate rows on ['PIN', 'SALEDATE', 'SALEPRIC']: 1397

NO_IMPRV top values:
 NO_IMPRV
NAN    217437
Y       20923
Name: count, dtype: int64

WARD top values:
 WARD
27    50262
26    25802
15    18545
25    15101
20    11867
Name: count, dtype: int64

SECTION top values:
 SECTION
10    19201
50    14816
20    13996
40    12859
30    12794
Name: count, dtype: int64


### Phase A - Light Cleaning (Real Property)

In [ ]:
import numpy as np

Coerce numeric columns and standardize keys to uppercase/trimmed strings.


In [ ]:
for c in ['PIN','BLOCKLOT','SALEPRIC','SALEDATE','NO_IMPRV','STRUCTAREA','CURRIMPR','BFCVIMPR','WARD','SECTION','NEIGHBOR']:
    if c not in real.columns:
        real[c] = np.nan

In [ ]:

for c in ['PIN','BLOCKLOT','WARD','SECTION','NEIGHBOR','NO_IMPRV']:
    real[c] = real[c].astype(str).str.strip().str.upper()

Numeric Conversion

In [ ]:

real['SALEPRIC']   = pd.to_numeric(real['SALEPRIC'], errors='coerce')
real['STRUCTAREA'] = pd.to_numeric(real['STRUCTAREA'], errors='coerce')
real['CURRIMPR']   = pd.to_numeric(real['CURRIMPR'], errors='coerce')
real['BFCVIMPR']   = pd.to_numeric(real['BFCVIMPR'], errors='coerce')

Parsing saledate

In [ ]:

sale_str = real['SALEDATE'].astype(str).str.extract(r'(\d+)')[0]
sale_str = sale_str.fillna('').str.zfill(8)
real['SALEDATE_PARSED'] = pd.to_datetime(sale_str, format='%m%d%Y', errors='coerce')

Identify duplicate keys/rows and collapse only safe duplicates.

In [ ]:

subset_cols = ['PIN','SALEDATE','SALEPRIC']
mask_dupegrp = real.duplicated(subset=subset_cols, keep=False)
dupes = real.loc[mask_dupegrp].copy()

print("Total rows in duplicate groups:", len(dupes))
print("Total distinct duplicate groups:", dupes[subset_cols].drop_duplicates().shape[0])


dupe_groups_preview = (
    dupes
    .groupby(subset_cols)
    .size()
    .reset_index(name='group_size')
    .sort_values('group_size', ascending=False)
    .head(10)
)
dupe_groups_preview

Total rows in duplicate groups: 2409
Total distinct duplicate groups: 1012


,PIN,SALEDATE,SALEPRIC,group_size
1005,PSC0010,11122019,0,67
1007,PSC0050,6132006,0,20
955,7140001,1011797,0,14
1011,PSC0085,1192021,0,14
1008,PSC0065,12101999,10587180,10
1010,PSC0075,12101999,10587180,10
331,3608016A,11021998,470000,9
1006,PSC0020,8272004,0,9
326,3559001,2071986,0,7
927,5391A001,4272010,0,6


In [ ]:

compare_cols = ['BLOCKLOT','NO_IMPRV','STRUCTAREA','CURRIMPR','BFCVIMPR','WARD','SECTION','NEIGHBOR']


distinct_counts = (
    dupes
    .groupby(subset_cols)[compare_cols]
    .nunique(dropna=False)
    .reset_index()
)


distinct_counts['all_identical'] = (distinct_counts[compare_cols] <= 1).all(axis=1)

print("Groups where all compare fields are identical:",
      int(distinct_counts['all_identical'].sum()))
print("Groups where something differs:",
      int((~distinct_counts['all_identical']).sum()))

diff_groups = distinct_counts.loc[~distinct_counts['all_identical']].head(10)
diff_groups


Groups where all compare fields are identical: 1012
Groups where something differs: 0


,PIN,SALEDATE,SALEPRIC,BLOCKLOT,NO_IMPRV,STRUCTAREA,CURRIMPR,BFCVIMPR,WARD,SECTION,NEIGHBOR,all_identical


In [ ]:
OUT_AUDIT = f"{DRIVE_FOLDER}/processed/duplicate_groups_for_review.csv"


keys_to_review = diff_groups[subset_cols]
suspect = dupes.merge(keys_to_review, on=subset_cols, how='inner')

suspect.to_csv(OUT_AUDIT, index=False)
print("Wrote an audit file with suspicious duplicate groups to:")
print(OUT_AUDIT)

suspect.head(20)

Wrote an audit file with suspicious duplicate groups to:
/content/drive/MyDrive/Capstone/processed/duplicate_groups_for_review.csv


,OBJECTID,PIN,PINRELATE,BLOCKLOT,BLOCK,LOT,WARD,SECTION,ASSESSOR,TAXBASE,...,STRUCTAREA,LDATE,OWNMDE,GRNDRENT,SUBTYPE_GEODB,SDATLINK,BLOCKPLAT,MAILTOADD,VACIND,SALEDATE_PARSED


In [ ]:

ignore_like = ["OBJECTID", "GLOBALID", "CREATED", "MODIFIED", "EDIT", "LOAD", "LDATE"]
cols_all = [c for c in real.columns if not any(k in c.upper() for k in ignore_like)]

sub = real[cols_all].copy()
subset_cols = ['PIN','SALEDATE','SALEPRIC']
mask_dupegrp = sub.duplicated(subset=subset_cols, keep=False)
dupes_all = sub.loc[mask_dupegrp]


all_nunique = dupes_all.groupby(subset_cols).nunique(dropna=False)
any_diff_groups = all_nunique.gt(1).any(axis=1).sum()

print("Groups with ANY difference across all checked columns:", any_diff_groups)



Groups with ANY difference across all checked columns: 52


In [ ]:
import pandas as pd
import numpy as np

subset_cols = ['PIN','SALEDATE','SALEPRIC']
mask_dupegrp = real.duplicated(subset=subset_cols, keep=False)
dupes = real.loc[mask_dupegrp].copy()


ignore_like = ["OBJECTID","GLOBALID","CREATED","CREATOR","EDIT","EDITOR",
               "MODIFIED","UPDATE","LOAD","LDATE","LOADDATE","LAST",
               "URL","LINK","SDAT","SHAPE","GEOM","X_","Y_","LONG","LAT"]

cols_all = [c for c in real.columns if not any(k in c.upper() for k in ignore_like)]


all_nunique = dupes.groupby(subset_cols)[cols_all].nunique(dropna=False)
groups_any_diff = all_nunique.gt(1).any(axis=1)

print("Duplicate groups total:", dupes[subset_cols].drop_duplicates().shape[0])
print("Groups with any diffs after ignoring volatile cols:", int(groups_any_diff.sum()))


diff_keys = all_nunique.index[groups_any_diff].to_frame(index=False)
diff_keys.head(10)


Duplicate groups total: 1012
Groups with any diffs after ignoring volatile cols: 52


,PIN,SALEDATE,SALEPRIC
0,0028060A,1011797,0
1,0601002B,1021998,1
2,0601002C,7022021,3500000
3,0601006E,1191987,0
4,1104043,10092015,0
5,1304002,10221991,0
6,1442008,12042014,2150000
7,1922301,10032023,1100000
8,1925A001,7192022,185000
9,1925A002,3042022,527500


In [ ]:

before = len(real)
real = real.drop_duplicates(subset=['PIN','SALEDATE','SALEPRIC'], keep='first')
after = len(real)
print(f"Dropped {before - after} duplicate rows using subset ['PIN','SALEDATE','SALEPRIC'].")

Dropped 1397 duplicate rows using subset ['PIN','SALEDATE','SALEPRIC'].


In [ ]:
real.shape

(236963, 80)

###**Parcel Data Cleaning**

In [ ]:
import geopandas as gpd

Loading the dataset

In [ ]:
gdf = gpd.read_file(parcel_df)

In [ ]:
gdf.shape

(223652, 16)

In [ ]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
gdf.columns

Index(['OBJECTID', 'TAG', 'LAST_ORG', 'CAPTURE_METHOD', 'LAST_USER',
       'EDIT_DATE', 'BLOCKNUM', 'PARCELNUM', 'PIN', 'CNTGRAPH', 'BLOCKLOT',
       'SUBTYPE_GEODB', 'GlobalID', 'SHAPE_Length', 'SHAPE_Area', 'geometry'],
      dtype='object')

In [ ]:
gdf.head(3)

,OBJECTID,TAG,LAST_ORG,CAPTURE_METHOD,LAST_USER,EDIT_DATE,BLOCKNUM,PARCELNUM,PIN,CNTGRAPH,BLOCKLOT,SUBTYPE_GEODB,GlobalID,SHAPE_Length,SHAPE_Area,geometry
0,1,600000000181720,100,BLK,None,1997-03-18 00:00:00+00:00,1786,024,1786024,24,1786 024,1,{4D15B0BE-8DCB-4569-9358-246ADCE1A1C1},0.000497,9.244674e-09,"MULTIPOLYGON (((-76.58335 39.28528, -76.58331 ..."
1,2,600000000022192,100,BLK,None,1997-03-18 00:00:00+00:00,4453,034,4453034,34,4453 034,1,{FDAFF0D9-E9DB-4CFB-8840-A659AFD7DBE3},0.001094,5.838560e-08,"MULTIPOLYGON (((-76.68029 39.35557, -76.68023 ..."
2,3,600000000071418,100,BLK,None,1997-03-18 00:00:00+00:00,3533,018,3533018,18,3533 018,1,{0139C8A7-4363-4DFE-83B2-6E332B91B2CB},0.000530,9.935054e-09,"MULTIPOLYGON (((-76.63013 39.32873, -76.63018 ..."


## Phase A : Light Cleaning

In [ ]:

for c in ['PIN','BLOCKLOT','SHAPE_Area','SHAPE_Length']:
    if c not in gdf.columns:
        gdf[c] = None

In [ ]:

print("Null counts (key fields):")
print(gdf[['PIN','BLOCKLOT','SHAPE_Area','SHAPE_Length']].isna().sum())

Null counts (key fields):
PIN             0
BLOCKLOT        2
SHAPE_Area      0
SHAPE_Length    0
dtype: int64


In [ ]:

dup_parc = gdf.duplicated(subset=['PIN','BLOCKLOT']).sum()
print("Duplicate key rows on ['PIN','BLOCKLOT']:", dup_parc)

Duplicate key rows on ['PIN','BLOCKLOT']: 500


In [ ]:

print("\nArea summary (ft²):")
print(gdf['SHAPE_Area'].astype(float).describe())
print("\nLength summary (ft):")
print(gdf['SHAPE_Length'].astype(float).describe())


Area summary (ft²):
count    2.236520e+05
mean     7.831293e-08
std      9.981754e-07
min      2.011062e-12
25%      1.076835e-08
50%      1.683027e-08
75%      3.663545e-08
max      2.599709e-04
Name: SHAPE_Area, dtype: float64

Length summary (ft):
count    223652.000000
mean          0.000934
std           0.001402
min           0.000017
25%           0.000579
50%           0.000730
75%           0.000982
max           0.174895
Name: SHAPE_Length, dtype: float64


In [ ]:
import numpy as np

Normalisation

In [ ]:

for c in ['PIN','BLOCKLOT']:
    gdf[c] = gdf[c].astype(str).str.strip().str.upper()

In [ ]:

gdf['SHAPE_Area']   = pd.to_numeric(gdf['SHAPE_Area'], errors='coerce')
gdf['SHAPE_Length'] = pd.to_numeric(gdf['SHAPE_Length'], errors='coerce')

In [ ]:

gdf = gdf.set_geometry(gdf.geometry.buffer(0))


gdf_m = gdf.to_crs(epsg=26918)
# Compute area (m² -> ft²) and length (m -> ft)
gdf['SHAPE_Area_ft2']  = gdf_m.area   * 10.7639
gdf['SHAPE_Length_ft'] = gdf_m.length * 3.28084

In [ ]:

parcel_area = (
    gdf[['PIN','BLOCKLOT','SHAPE_Area_ft2','SHAPE_Length_ft']]
    .drop_duplicates(subset=['PIN','BLOCKLOT'])
    .copy()
)

print("parcel_area rows:", len(parcel_area))
parcel_area.head(3)

parcel_area rows: 223152


,PIN,BLOCKLOT,SHAPE_Area_ft2,SHAPE_Length_ft
0,1786024,1786 024,952.831518,173.515613
1,4453034,4453 034,6012.093040,340.489735
2,3533018,3533 018,1023.386873,185.571346


In [ ]:

if gdf.crs is None:

    gdf.set_crs(epsg=4326, inplace=True)
elif gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(epsg=4326)

In [ ]:
centroids = gdf.geometry.centroid
gdf['lon'] = centroids.x
gdf['lat'] = centroids.y

/tmp/ipython-input-3633905687.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = gdf.geometry.centroid


removing duplicates

In [ ]:
gdf.shape

(223652, 20)

In [ ]:
dup_mask = gdf.duplicated(subset=['PIN','BLOCKLOT'], keep=False)
dupes = gdf.loc[dup_mask].copy()

print("Total rows in duplicate groups:", len(dupes))
print("Distinct duplicate keys:", dupes[['PIN','BLOCKLOT']].drop_duplicates().shape[0])

grp = dupes.groupby(['PIN','BLOCKLOT']).agg(
    n_rows=('PIN','size'),
    area_min=('SHAPE_Area_ft2','min'),
    area_max=('SHAPE_Area_ft2','max'),
    len_min =('SHAPE_Length_ft','min'),
    len_max =('SHAPE_Length_ft','max')
).reset_index()

ABS_TOL = 1.0         # 1 sq ft absolute tolerance
REL_TOL = 0.0001      # 0.01% relative tolerance

def is_close(minv, maxv):
    if pd.isna(minv) or pd.isna(maxv):
        return False
    abs_ok = (maxv - minv) <= ABS_TOL
    rel_ok = ((maxv - minv) / max(1.0, maxv)) <= REL_TOL
    return abs_ok or rel_ok

grp['area_close'] = grp.apply(lambda r: is_close(r['area_min'], r['area_max']), axis=1)
grp['len_close']  = grp.apply(lambda r: is_close(r['len_min'],  r['len_max']),  axis=1)

# Classify: safe_to_collapse if both area and length close
grp['safe_to_collapse'] = grp['area_close'] & grp['len_close']

safe_keys   = grp.loc[grp['safe_to_collapse'], ['PIN','BLOCKLOT']]
review_keys = grp.loc[~grp['safe_to_collapse'], ['PIN','BLOCKLOT']]

print("Duplicate keys SAFE to collapse:", len(safe_keys))
print("Duplicate keys that need REVIEW:", len(review_keys))

# Keep these handy DataFrames for the next cell
dupe_groups_summary = grp
dupe_groups_summary.head(10)


Total rows in duplicate groups: 760
Distinct duplicate keys: 260
Duplicate keys SAFE to collapse: 0
Duplicate keys that need REVIEW: 260


,PIN,BLOCKLOT,n_rows,area_min,area_max,len_min,len_max,area_close,len_close,safe_to_collapse
0,,NONE,2,495.068770,1.234707e+03,263.948665,482.746454,False,False,False
1,0433001,0433 001,2,104246.123427,2.432647e+05,1532.604196,2264.658794,False,False,False
2,0469018,0469 018,2,891.342037,1.665280e+03,203.491943,530.173215,False,False,False
3,0501004,0501 004,2,8012.797213,1.637094e+04,365.341759,506.034001,False,False,False
4,0577025,0577 025,2,4411.517071,5.544671e+03,278.080178,300.322777,False,False,False
5,0616001,0616 001,2,14960.803694,3.624308e+04,615.666974,802.577006,False,False,False
6,0682001,0682 001,2,286558.500118,3.205400e+05,2275.832993,2295.939548,False,False,False
7,0688A001,0688A001,3,13144.231425,1.188423e+06,535.995661,6791.785965,False,False,False
8,0731001,0731 001,3,270971.364358,3.022195e+06,2334.945691,10652.913393,False,False,False
9,1030058,1030 058,2,998.764074,1.603893e+03,184.116208,291.348002,False,False,False


In [ ]:
gdf_dedup = (
    gdf
    .sort_values(["PIN","BLOCKLOT","SHAPE_Area_ft2","SHAPE_Length_ft"], ascending=[True, True, False, False])
    .drop_duplicates(subset=["PIN","BLOCKLOT"], keep="first")
    .reset_index(drop=True)
)

In [ ]:
gdf_dedup.shape

(223152, 20)

### **Joining process**

In [ ]:

real_keys = real[['PIN','BLOCKLOT']].copy()


pin_join = real[['PIN']].merge(gdf_dedup[['PIN','SHAPE_Area_ft2']], on='PIN', how='left')
pin_matches = int(pin_join['SHAPE_Area_ft2'].notna().sum())

# Match via BLOCKLOT
blk_join = real[['BLOCKLOT']].merge(gdf_dedup[['BLOCKLOT','SHAPE_Area_ft2']], on='BLOCKLOT', how='left')
blk_matches = int(blk_join['SHAPE_Area_ft2'].notna().sum())

print("Matches via PIN     :", pin_matches)
print("Matches via BLOCKLOT:", blk_matches)

# Choose the better key automatically
chosen_key = 'PIN' if pin_matches >= blk_matches else 'BLOCKLOT'
print("Chosen join key ->", chosen_key)

Matches via PIN     : 189231
Matches via BLOCKLOT: 216783
Chosen join key -> BLOCKLOT


# Joining the two dataset using PIN & BLOCKLOT

In [ ]:
# 1) Slim, unique parcel table on both keys
cols = ['PIN','BLOCKLOT','SHAPE_Area_ft2','SHAPE_Length_ft']
if 'lon' in gdf_dedup.columns: cols += ['lon','lat']
parcel_slim = gdf_dedup[cols].drop_duplicates(subset=['PIN','BLOCKLOT'])

# 2) Join by PIN
j_pin = real.merge(
    parcel_slim[['PIN','SHAPE_Area_ft2','SHAPE_Length_ft'] + (['lon','lat'] if 'lon' in parcel_slim.columns else [])],
    on='PIN', how='left'
).rename(columns={
    'SHAPE_Area_ft2': 'SHAPE_Area_pin',
    'SHAPE_Length_ft': 'SHAPE_Length_pin',
    **({'lon':'lon_pin','lat':'lat_pin'} if 'lon' in parcel_slim.columns else {})
})

# 3) Join by BLOCKLOT
j_blk = j_pin.merge(
    parcel_slim[['BLOCKLOT','SHAPE_Area_ft2','SHAPE_Length_ft'] + (['lon','lat'] if 'lon' in parcel_slim.columns else [])],
    on='BLOCKLOT', how='left'
).rename(columns={
    'SHAPE_Area_ft2': 'SHAPE_Area_blk',
    'SHAPE_Length_ft': 'SHAPE_Length_blk',
    **({'lon':'lon_blk','lat':'lat_blk'} if 'lon' in parcel_slim.columns else {})
})

# 4) Coalesce: prefer PIN match, else BLOCKLOT
joined = j_blk.copy()
joined['SHAPE_Area']   = joined['SHAPE_Area_pin'].combine_first(joined['SHAPE_Area_blk'])
joined['SHAPE_Length'] = joined['SHAPE_Length_pin'].combine_first(joined['SHAPE_Length_blk'])
if 'lon_pin' in joined.columns:
    joined['lon'] = joined['lon_pin'].combine_first(joined['lon_blk'])
    joined['lat'] = joined['lat_pin'].combine_first(joined['lat_blk'])

# Which key supplied the area?
joined['join_key_used'] = np.where(
    joined['SHAPE_Area_pin'].notna(), 'PIN',
    np.where(joined['SHAPE_Area_blk'].notna(), 'BLOCKLOT', 'NONE')
)

# 5) Cleanup helper columns
drop_cols = [c for c in joined.columns if c.endswith('_pin') or c.endswith('_blk')]
joined.drop(columns=drop_cols, inplace=True, errors='ignore')

print("Joined rows:", len(joined))
print("Rows with SHAPE_Area:", int(joined['SHAPE_Area'].notna().sum()))
print("Rows missing SHAPE_Area:", int(joined['SHAPE_Area'].isna().sum()))
joined.head(3)


Joined rows: 236989
Rows with SHAPE_Area: 217208
Rows missing SHAPE_Area: 19781


,OBJECTID,PIN,PINRELATE,BLOCKLOT,BLOCK,LOT,WARD,SECTION,ASSESSOR,TAXBASE,...,SDATLINK,BLOCKPLAT,MAILTOADD,VACIND,SALEDATE_PARSED,SHAPE_Area,SHAPE_Length,lon,lat,join_key_used
0,1,1001,1001,0001 001,1,1,15,370,329,172000,...,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"1639 MORELAND AVE, 21216",NaN,2023-05-22,1359.595646,200.090945,-76.651146,39.309421,BLOCKLOT
1,2,1002,1002,0001 002,1,2,15,370,329,49500,...,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"1639 MORELAND AVE, 21216",NaN,2023-05-22,1173.811654,195.648745,-76.651093,39.309424,BLOCKLOT
2,3,1003,1003,0001 003,1,3,15,370,377,9000,...,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"HILTON PLAZA STE 201, 21208",Y,1975-10-17,1188.010638,196.006763,-76.651043,39.309426,BLOCKLOT


In [ ]:
import numpy as np

total = len(joined)
matched = int(joined['SHAPE_Area'].notna().sum())
missing = total - matched
print(f"Total rows: {total} | Matched area: {matched} ({matched/total:.1%}) | Missing area: {missing} ({missing/total:.1%})")

# A) Which key drove the matches?
print("\njoin_key_used counts:")
print(joined['join_key_used'].value_counts(dropna=False))

# B) Anti-join diagnostics (who failed to match by each key)
#    Real vs Parcel by PIN
anti_pin = real[['PIN']].merge(gdf_dedup[['PIN']], on='PIN', how='left', indicator=True)
anti_pin = anti_pin[anti_pin['_merge']=='left_only'].drop(columns=['_merge']).drop_duplicates()
print("\nUnique real rows with PIN not found in parcel:", len(anti_pin))

#    Real vs Parcel by BLOCKLOT
anti_blk = real[['BLOCKLOT']].merge(gdf_dedup[['BLOCKLOT']], on='BLOCKLOT', how='left', indicator=True)
anti_blk = anti_blk[anti_blk['_merge']=='left_only'].drop(columns=['_merge']).drop_duplicates()
print("Unique real rows with BLOCKLOT not found in parcel:", len(anti_blk))

# C) Conflicting matches: both keys hit but areas differ (rare; review a few)
if 'SHAPE_Area_pin' in j_blk.columns and 'SHAPE_Area_blk' in j_blk.columns:
    both_hit = j_blk['SHAPE_Area_pin'].notna() & j_blk['SHAPE_Area_blk'].notna()
    conflict = both_hit & (j_blk['SHAPE_Area_pin'] != j_blk['SHAPE_Area_blk'])
    print("\nRows where both keys matched but areas differ:", int(conflict.sum()))
    if conflict.any():
        display(j_blk.loc[conflict, ['PIN','BLOCKLOT','SHAPE_Area_pin','SHAPE_Area_blk']].head(10))

# D) Area sanity: zeros/negatives and distribution
print("\nZero/negative SHAPE_Area rows:", int((joined['SHAPE_Area']<=0).sum()))
if joined['SHAPE_Area'].notna().any():
    print("\nSHAPE_Area describe (ft²):")
    print(joined['SHAPE_Area'].describe(percentiles=[.01,.05,.25,.5,.75,.95,.99]))


Total rows: 236989 | Matched area: 217208 (91.7%) | Missing area: 19781 (8.3%)

join_key_used counts:
join_key_used
PIN         189244
BLOCKLOT     27964
NONE         19781
Name: count, dtype: int64

Unique real rows with PIN not found in parcel: 47438
Unique real rows with BLOCKLOT not found in parcel: 19890

Rows where both keys matched but areas differ: 25


,PIN,BLOCKLOT,SHAPE_Area_pin,SHAPE_Area_blk
34997,1031052,1031 052,706.270220,1059.596650
37045,1120059,1120 059,1402.426406,1400.618237
40625,1335020,1335 020,2773.467846,5161.452398
58192,1731054,1731 054,1054.939668,1115.761471
67940,1882051,1882 051,898.126531,944.801049
69846,1922301,1922 301,1741.473933,1796.931887
72075,2002B071A,2002B071A,9821.831734,2961.040010
73250,2024001,2024 001,1084.634822,1020.470415
75014,2134042,2134 042,16135.004669,38360.263953
78746,2244A041,2244A041,72764.678949,29510.001833



Zero/negative SHAPE_Area rows: 0

SHAPE_Area describe (ft²):
count    2.172080e+05
mean     7.569279e+03
std      9.761118e+04
min      2.028959e-01
1%       4.178131e+02
5%       7.271489e+02
25%      1.107309e+03
50%      1.726607e+03
75%      3.725763e+03
95%      1.323120e+04
99%      7.895166e+04
max      2.678151e+07
Name: SHAPE_Area, dtype: float64


In [ ]:
joined.duplicated(subset=['PIN','BLOCKLOT']).sum()

np.int64(331)

In [ ]:
import pandas as pd
import numpy as np

df = joined.copy()  # keep original safe

# Ensure expected columns exist
for c in ['SALEPRIC','STRUCTAREA','CURRIMPR','BFCVIMPR','SHAPE_Area','SHAPE_Length','NO_IMPRV','SALEDATE','SALEDATE_PARSED']:
    if c not in df.columns: df[c] = np.nan

# Numeric conversions
df['SALEPRIC']    = pd.to_numeric(df['SALEPRIC'], errors='coerce')
df['STRUCTAREA']  = pd.to_numeric(df['STRUCTAREA'], errors='coerce')
df['CURRIMPR']    = pd.to_numeric(df['CURRIMPR'], errors='coerce')
df['BFCVIMPR']    = pd.to_numeric(df['BFCVIMPR'], errors='coerce')
df['SHAPE_Area']  = pd.to_numeric(df['SHAPE_Area'], errors='coerce')
df['SHAPE_Length']= pd.to_numeric(df['SHAPE_Length'], errors='coerce')

# Parse SALEDATE if parsed not present
if 'SALEDATE_PARSED' not in df.columns or df['SALEDATE_PARSED'].isna().all():
    sale_str = df['SALEDATE'].astype(str).str.extract(r'(\d+)')[0].fillna('').str.zfill(8)
    df['SALEDATE_PARSED'] = pd.to_datetime(sale_str, format='%m%d%Y', errors='coerce')

# Time features
df['SALEYEAR']  = df['SALEDATE_PARSED'].dt.year
df['SALEMONTH'] = df['SALEDATE_PARSED'].dt.month
df['SALEQTR']   = df['SALEDATE_PARSED'].dt.to_period('Q').astype(str)

# Size helpers
df['LOT_ACRES'] = df['SHAPE_Area'] / 43560.0

print("Rows:", len(df))
print("Parsed dates %:", round(100*df['SALEDATE_PARSED'].notna().mean(), 1))
print("Positive area %:", round(100*(df['SHAPE_Area']>0).mean(), 1))
df.head(3)

Rows: 236989
Parsed dates %: 100.0
Positive area %: 91.7


,OBJECTID,PIN,PINRELATE,BLOCKLOT,BLOCK,LOT,WARD,SECTION,ASSESSOR,TAXBASE,...,SALEDATE_PARSED,SHAPE_Area,SHAPE_Length,lon,lat,join_key_used,SALEYEAR,SALEMONTH,SALEQTR,LOT_ACRES
0,1,1001,1001,0001 001,1,1,15,370,329,172000,...,2023-05-22,1359.595646,200.090945,-76.651146,39.309421,BLOCKLOT,2023.0,5.0,2023Q2,0.031212
1,2,1002,1002,0001 002,1,2,15,370,329,49500,...,2023-05-22,1173.811654,195.648745,-76.651093,39.309424,BLOCKLOT,2023.0,5.0,2023Q2,0.026947
2,3,1003,1003,0001 003,1,3,15,370,377,9000,...,1975-10-17,1188.010638,196.006763,-76.651043,39.309426,BLOCKLOT,1975.0,10.0,1975Q4,0.027273


In [ ]:
# Vacant land heuristic
land_only = (
    (df['NO_IMPRV'].astype(str).str.strip().str.upper() == 'Y') &
    (df['STRUCTAREA'].fillna(0) == 0) &
    (df['CURRIMPR'].fillna(0) == 0) &
    (df['BFCVIMPR'].fillna(0) == 0)
)

# Valid sale & area
valid_sale = (df['SALEPRIC'] > 0) & df['SALEDATE_PARSED'].notna()
valid_area = df['SHAPE_Area'] > 0

# Optional: restrict to recent market (edit if you want a different window)
START_DATE = pd.Timestamp(2019,1,1)
recent = df['SALEDATE_PARSED'] >= START_DATE

df_base = df.loc[land_only & valid_sale & valid_area & recent].copy()
print("Base rows after filters:", len(df_base))
df_base[['NO_IMPRV','STRUCTAREA','CURRIMPR','BFCVIMPR','SALEPRIC','SHAPE_Area','SALEDATE_PARSED']].head(5)


Base rows after filters: 2079


,NO_IMPRV,STRUCTAREA,CURRIMPR,BFCVIMPR,SALEPRIC,SHAPE_Area,SALEDATE_PARSED
1192,Y,0.0,0,0,23776,959.957615,2022-03-09
1196,Y,0.0,0,0,7427,1059.027952,2019-10-16
1488,Y,0.0,0,0,500,1263.554884,2023-01-19
1753,Y,0.0,0,0,100000,3531.429975,2022-10-13
1810,Y,0.0,0,0,1000,5823.224993,2019-04-24


In [ ]:
# Target
df_base['price_per_sqft'] = (df_base['SALEPRIC'] / df_base['SHAPE_Area']).round(2)

# Remove nonsense and extreme tails
df_base = df_base[np.isfinite(df_base['price_per_sqft']) & (df_base['price_per_sqft'] > 0)].copy()

# Gentle winsorization at 1st–99th percentiles
if len(df_base) > 0:
    lo, hi = df_base['price_per_sqft'].quantile([0.01, 0.99])
    df_base = df_base[(df_base['price_per_sqft'] >= lo) & (df_base['price_per_sqft'] <= hi)].copy()

print("Rows after ppsf cleanup:", len(df_base))
print("ppsf median:", round(df_base['price_per_sqft'].median(), 2))
df_base[['SALEPRIC','SHAPE_Area','price_per_sqft']].head(5)


Rows after ppsf cleanup: 2042
ppsf median: 17.12


,SALEPRIC,SHAPE_Area,price_per_sqft
1192,23776,959.957615,24.77
1196,7427,1059.027952,7.01
1488,500,1263.554884,0.40
1753,100000,3531.429975,28.32
1810,1000,5823.224993,0.17


In [ ]:
sale_key = [c for c in ['PIN','BLOCKLOT','SALEDATE','SALEPRIC'] if c in df_base.columns]

dupes = df_base.duplicated(subset=sale_key).sum() if sale_key else 0
print("Duplicate sale identities:", dupes)

if dupes > 0:
    before = len(df_base)
    df_base = df_base.drop_duplicates(subset=sale_key, keep='first')
    print(f"Dropped {before - len(df_base)} duplicates. Final rows: {len(df_base)}")


Duplicate sale identities: 0


In [ ]:
if {'lon','lat'}.issubset(df_base.columns):
    # Baltimore-ish bounding box (rough): lon ∈ [-76.8, -76.4], lat ∈ [39.18, 39.40]
    in_box = (
        df_base['lon'].between(-76.8, -76.4, inclusive='both') &
        df_base['lat'].between(39.18, 39.40, inclusive='both')
    )
    kept_before = len(df_base)
    df_base = df_base.loc[in_box].copy()
    print(f"Coordinate sanity filter kept {len(df_base)} / {kept_before} rows.")
else:
    print("No lon/lat in data — skipping coordinate sanity check.")


Coordinate sanity filter kept 2042 / 2042 rows.


In [ ]:
# Keep a tidy set of columns for EDA (add/remove as needed)
eda_cols = [
    'PIN','BLOCKLOT','SALEDATE','SALEDATE_PARSED','SALEYEAR','SALEMONTH','SALEQTR',
    'SALEPRIC','SHAPE_Area','LOT_ACRES','SHAPE_Length','price_per_sqft',
    'WARD','SECTION','NEIGHBOR','join_key_used'
]
if {'lon','lat'}.issubset(df_base.columns):
    eda_cols += ['lon','lat']

eda_df = df_base[[c for c in eda_cols if c in df_base.columns]].copy()

print("EDA-ready rows:", len(eda_df))
print("\nNull % (selected cols):")
print((eda_df.isna().mean() * 100).round(1)[['SALEPRIC','SHAPE_Area','price_per_sqft','WARD','SECTION','NEIGHBOR']].sort_values(ascending=False))

print("\nSALEYEAR counts:")
print(eda_df['SALEYEAR'].value_counts().sort_index())

print("\nprice_per_sqft describe:")
print(eda_df['price_per_sqft'].describe(percentiles=[.01,.05,.25,.5,.75,.95,.99]).round(2))

eda_df.head(5)


EDA-ready rows: 2042

Null % (selected cols):
SALEPRIC          0.0
SHAPE_Area        0.0
price_per_sqft    0.0
WARD              0.0
SECTION           0.0
NEIGHBOR          0.0
dtype: float64

SALEYEAR counts:
SALEYEAR
2019.0    353
2020.0    170
2021.0    285
2022.0    363
2023.0    344
2024.0    290
2025.0    237
Name: count, dtype: int64

price_per_sqft describe:
count    2042.00
mean       99.13
std       240.20
min         0.01
1%          0.01
5%          0.35
25%         3.19
50%        17.12
75%        69.80
95%       525.85
99%      1383.26
max      1791.48
Name: price_per_sqft, dtype: float64


,PIN,BLOCKLOT,SALEDATE,SALEDATE_PARSED,SALEYEAR,SALEMONTH,SALEQTR,SALEPRIC,SHAPE_Area,LOT_ACRES,SHAPE_Length,price_per_sqft,WARD,SECTION,NEIGHBOR,join_key_used,lon,lat
1192,20052,0020 052,3092022,2022-03-09,2022.0,3.0,2022Q1,23776,959.957615,0.022038,183.981679,24.77,15,80,SANDTOWN-WINCHESTER,BLOCKLOT,-76.646824,39.305613
1196,20056,0020 056,10162019,2019-10-16,2019.0,10.0,2019Q4,7427,1059.027952,0.024312,186.475393,7.01,15,80,SANDTOWN-WINCHESTER,BLOCKLOT,-76.646830,39.305747
1488,26021,0026 021,1192023,2023-01-19,2023.0,1.0,2023Q1,500,1263.554884,0.029007,191.582464,0.40,15,80,SANDTOWN-WINCHESTER,BLOCKLOT,-76.646463,39.305177
1753,32001,0032 001,10132022,2022-10-13,2022.0,10.0,2022Q4,100000,3531.429975,0.081070,269.936948,28.32,15,90,SANDTOWN-WINCHESTER,BLOCKLOT,-76.644914,39.303309
1810,32060,0032 060,4242019,2019-04-24,2019.0,4.0,2019Q2,1000,5823.224993,0.133683,337.632491,0.17,15,90,SANDTOWN-WINCHESTER,BLOCKLOT,-76.645236,39.303717


In [ ]:
OUT_FILE = f"{out_dir}/final_vacant_land_eda.csv"
eda_df.to_csv(OUT_FILE, index=False)
print(f"EDA dataframe saved to: {OUT_FILE}")

EDA dataframe saved to: /content/drive/MyDrive/Capstone/processed/final_vacant_land_eda.csv


### **Exploratory Data Analysis**

In [ ]:
import plotly.express as px

In [ ]:
fig = px.histogram(
    eda_df, x="price_per_sqft", nbins=60,
    title="Distribution of Price per Square Foot (trimmed 1–99th pct)",
    labels={"price_per_sqft":"Price per ft² (USD)"}
)
fig.update_traces(marker_line_width=0.2)
fig.show()


In [ ]:
ts = (
    eda_df.groupby('SALEQTR', as_index=False)
          .agg(median_ppsf=('price_per_sqft','median'),
               n_sales=('price_per_sqft','size'))
          .sort_values('SALEQTR')
)

fig = px.line(
    ts, x="SALEQTR", y="median_ppsf",
    title="Quarterly Median Price per ft²",
    markers=True, labels={"SALEQTR":"Quarter","median_ppsf":"Median ppsf (USD)"}
)

fig2 = px.bar(ts, x="SALEQTR", y="n_sales", labels={"n_sales":"Sales count"})
fig.show()
fig2.show()


In [ ]:



fig = px.scatter(
    eda_df, x="SHAPE_Area", y="price_per_sqft",
    trendline="ols",
    hover_data=["PIN","BLOCKLOT","SALEYEAR","NEIGHBOR"] if "NEIGHBOR" in eda_df.columns else ["PIN","BLOCKLOT","SALEYEAR"],
    title="Lot Area vs Price per ft² (log–log) with OLS Trendline",
    labels={"SHAPE_Area":"Lot area (ft²)", "price_per_sqft":"Price per ft² (USD)"}
)
fig.update_xaxes(type="log")
fig.update_yaxes(type="log")
fig.show()

In [ ]:
if "WARD" in eda_df.columns:

    top_wards = (
        eda_df['WARD'].astype(str).value_counts()
        .head(12).index.tolist()
    )
    sub = eda_df[eda_df['WARD'].astype(str).isin(top_wards)].copy()
    fig = px.box(
        sub, x="WARD", y="price_per_sqft",
        points="suspectedoutliers", title="Price per ft² by WARD (Top 12 by volume)",
        labels={"price_per_sqft":"Price per ft² (USD)"}
    )
    fig.show()
else:
    print("WARD not found; skipping boxplot.")


In [ ]:
if "NEIGHBOR" in eda_df.columns:
    top_neigh = (
        eda_df['NEIGHBOR'].astype(str).value_counts()
        .head(20).index.tolist()
    )
    sub = eda_df[eda_df['NEIGHBOR'].astype(str).isin(top_neigh)].copy()
    grp = sub.groupby('NEIGHBOR', as_index=False).agg(
        median_ppsf=('price_per_sqft','median'),
        n=('price_per_sqft','size')
    ).sort_values(['median_ppsf'], ascending=False)

    fig = px.bar(
        grp, x="NEIGHBOR", y="median_ppsf",
        hover_data=["n"],
        title="Median Price per ft² by Neighborhood (Top 20 by count)",
        labels={"median_ppsf":"Median ppsf (USD)"},
    )
    fig.update_xaxes(tickangle=45)
    fig.show()
else:
    print("NEIGHBOR not found; skipping neighborhood chart.")


In [ ]:
if {"lon","lat"}.issubset(eda_df.columns):
    fig = px.scatter_mapbox(
        eda_df.sample(min(len(eda_df), 15000), random_state=42),
        lat="lat", lon="lon", color="price_per_sqft", size=None,
        color_continuous_scale="Viridis",
        zoom=10, height=600,
        hover_name="NEIGHBOR" if "NEIGHBOR" in eda_df.columns else "PIN",
        hover_data={"price_per_sqft":":.2f","SALEYEAR":True,"PIN":True,"BLOCKLOT":True}
    )
    fig.update_layout(mapbox_style="carto-positron", title="Sales Map Colored by Price per ft²")
    fig.show()
else:
    print("No lon/lat; skipping map.")


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

OLS Model

In [ ]:
# Drop missing target or area
eda_df = eda_df.dropna(subset=["price_per_sqft", "SHAPE_Area"])

# Remove non-positive values
eda_df = eda_df[(eda_df["price_per_sqft"] > 0) & (eda_df["SHAPE_Area"] > 0)]

# Log transform (standard for land valuation)
eda_df["log_ppsf"] = np.log(eda_df["price_per_sqft"])
eda_df["log_area"] = np.log(eda_df["SHAPE_Area"])

eda_df["SALEDATE"] = pd.to_datetime(eda_df["SALEDATE"])
eda_df["year"] = eda_df["SALEDATE"].dt.year
eda_df["quarter"] = eda_df["SALEDATE"].dt.to_period("Q").astype(str)


for col in ["WARD", "SECTION", "NEIGHBOR", "year"]:
    if col in eda_df.columns:
        eda_df[col] = eda_df[col].astype("category")


formula = """
log_ppsf ~
    log_area +
    C(WARD) +
    C(SECTION) +
    C(year)
"""

ols_model = smf.ols(formula=formula, data=eda_df).fit()

print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:               log_ppsf   R-squared:                       0.274
Model:                            OLS   Adj. R-squared:                  0.242
Method:                 Least Squares   F-statistic:                     8.680
Date:                Fri, 21 Nov 2025   Prob (F-statistic):           8.47e-86
Time:                        00:56:57   Log-Likelihood:                -4262.7
No. Observations:                2042   AIC:                             8697.
Df Residuals:                    1956   BIC:                             9181.
Df Model:                          85                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            10.6053      1.05

Improved OLS With Only Required Outputs

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, mean_absolute_error


eda_df = eda_df.dropna(subset=["price_per_sqft", "SHAPE_Area"])
eda_df = eda_df[(eda_df["price_per_sqft"] > 0) & (eda_df["SHAPE_Area"] > 0)]

eda_df["log_ppsf"] = np.log(eda_df["price_per_sqft"])
eda_df["log_area"] = np.log(eda_df["SHAPE_Area"])
eda_df["log_area_sq"] = eda_df["log_area"] ** 2

eda_df["SALEDATE"] = pd.to_datetime(eda_df["SALEDATE"])
eda_df["year"] = eda_df["SALEDATE"].dt.year.astype("category")
eda_df["quarter"] = eda_df["SALEDATE"].dt.quarter.astype("category")

if "NEIGHBOR" in eda_df.columns:
    counts = eda_df["NEIGHBOR"].value_counts()
    small_groups = counts[counts < 10].index
    eda_df["NEIGHBOR_GRP"] = eda_df["NEIGHBOR"].replace(small_groups, "OTHER")
    eda_df["NEIGHBOR_GRP"] = eda_df["NEIGHBOR_GRP"].astype("category")
else:
    eda_df["NEIGHBOR_GRP"] = "OTHER"

formula = """
log_ppsf ~
    log_area +
    log_area_sq +
    C(WARD) +
    C(NEIGHBOR_GRP) +
    C(year) +
    C(quarter)
"""

model = smf.ols(formula=formula, data=eda_df).fit()

pred_log = model.predict(eda_df)

pred_ppsf = np.exp(pred_log)
actual_ppsf = eda_df["price_per_sqft"]

r2 = model.rsquared
adj_r2 = model.rsquared_adj

rmse = np.sqrt(mean_squared_error(actual_ppsf, pred_ppsf))

mae = mean_absolute_error(actual_ppsf, pred_ppsf)

mape = (np.abs((actual_ppsf - pred_ppsf) / actual_ppsf)).mean() * 100

print("=== OLS Model Performance ===")
print(f"R-squared: {r2:.4f}")
print(f"Adjusted R-squared: {adj_r2:.4f}")
print(f"RMSE (on price_per_sqft): {rmse:.4f}")
print(f"MAE (on price_per_sqft): {mae:.4f}")
print(f"MAPE: {mape:.2f}%")


/tmp/ipython-input-3455698458.py:21: FutureWarning:

The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.



=== OLS Model Performance ===
R-squared: 0.3404
Adjusted R-squared: 0.3110
RMSE (on price_per_sqft): 348.5351
MAE (on price_per_sqft): 85.9425
MAPE: 1005.97%


In [ ]:
import numpy as np

# 1. Predictions in log-space
pred_log = model.predict(eda_df)

# 2. Residuals in log-space
residuals = eda_df["log_ppsf"] - pred_log

# 3. Duan smearing factor
smearing_factor = np.exp(residuals).mean()

# 4. Corrected predictions in price_per_sqft
pred_ppsf = np.exp(pred_log) * smearing_factor

# 5. Actual values
actual_ppsf = eda_df["price_per_sqft"]

# 6. Metrics
rmse = np.sqrt(mean_squared_error(actual_ppsf, pred_ppsf))
mae = mean_absolute_error(actual_ppsf, pred_ppsf)
mape = (np.abs(actual_ppsf - pred_ppsf) / actual_ppsf).mean() * 100

print("=== Corrected OLS Model Performance ===")
print(f"R-squared: {r2:.4f}")
print(f"Adjusted R-squared: {adj_r2:.4f}")
print(f"Smearing Factor: {smearing_factor:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MAPE: {mape:.2f}%")

=== Corrected OLS Model Performance ===
R-squared: 0.3404
Adjusted R-squared: 0.3110
Smearing Factor: 7.1052
RMSE: 2121.1495
MAE: 258.0702
MAPE: 7298.54%


performance values are unextremely high

checking the extreme values

In [ ]:
eda_df["price_per_sqft"].describe(percentiles=[0.99, 0.999])

,price_per_sqft
count,2042.000000
mean,99.134878
std,240.198772
min,0.010000
50%,17.125000
99%,1383.258800
99.9%,1725.206170
max,1791.480000


Check top 20 largest price/ft²

In [ ]:
eda_df[["SALEPRIC", "SHAPE_Area", "price_per_sqft"]].sort_values("price_per_sqft", ascending=False).head(20)


,SALEPRIC,SHAPE_Area,price_per_sqft
21745,2256110,1259.357694,1791.48
93918,20650000,11817.165124,1747.46
45263,79000,45.760582,1726.38
164037,357000,210.278840,1697.75
30941,425000,255.518042,1663.29
236205,375000,226.830942,1653.21
159266,5349000,3239.537648,1651.16
36278,1100000,676.217579,1626.70
145109,16100000,10032.747254,1604.74
211104,8277500,5303.806998,1560.67


Check if tiny parcels exist

In [ ]:
eda_df[eda_df["SHAPE_Area"] < 100].head(10)

,PIN,BLOCKLOT,SALEDATE,SALEDATE_PARSED,SALEYEAR,SALEMONTH,SALEQTR,SALEPRIC,SHAPE_Area,LOT_ACRES,...,NEIGHBOR,join_key_used,lon,lat,log_ppsf,log_area,year,quarter,log_area_sq,NEIGHBOR_GRP
45253,1476F031A,1476F031A,1970-01-01 00:00:00.001062020,2020-01-06,2020.0,1.0,2020Q1,65000,46.743117,0.001073,...,BEREA,PIN,-76.573607,39.309736,7.237476,3.844667,1970,1,14.781464,BEREA
45259,1476F034A,1476F034A,1970-01-01 00:00:00.008012019,2019-08-01,2019.0,8.0,2019Q3,40000,46.058004,0.001057,...,BEREA,PIN,-76.573397,39.309738,6.766733,3.829902,1970,1,14.668146,BEREA
45263,1476F036A,1476F036A,1970-01-01 00:00:00.004292021,2021-04-29,2021.0,4.0,2021Q2,79000,45.760582,0.001051,...,BEREA,PIN,-76.573257,39.309739,7.453782,3.823423,1970,1,14.618564,BEREA
196555,6019P062,6019P062,1970-01-01 00:00:00.009192024,2024-09-19,2024.0,9.0,2024Q3,5000,8.046181,0.000185,...,FRANKFORD,PIN,-76.529714,39.336849,6.431991,2.085198,1970,1,4.348049,FRANKFORD


In [ ]:
df_c= eda_df[
    (eda_df["SHAPE_Area"] >= 200) &
    (eda_df["price_per_sqft"] <= 1000)
]


In [ ]:
df_c[["SALEPRIC","SHAPE_Area","price_per_sqft"]].sort_values("price_per_sqft",ascending=False).head(20)


,SALEPRIC,SHAPE_Area,price_per_sqft
29651,225000,236.802900,950.16
68376,385000,408.962704,941.41
160554,1240000,1322.997817,937.27
115143,232500,250.008622,929.97
29558,270000,294.691539,916.21
34542,396000,439.737688,900.54
29648,192000,213.329439,900.02
34541,396000,446.508896,886.88
40315,185000,212.762378,869.51
36143,515000,604.688630,851.68


In [ ]:
import numpy as np

# 1. Predictions in log-space
pred_log = model.predict(df_c)

# 2. Residuals in log-space
residuals = df_c["log_ppsf"] - pred_log

# 3. Duan smearing factor
smearing_factor = np.exp(residuals).mean()

# 4. Corrected predictions in price_per_sqft
pred_ppsf = np.exp(pred_log) * smearing_factor

# 5. Actual values
actual_ppsf = df_c["price_per_sqft"]

# 6. Metrics
rmse = np.sqrt(mean_squared_error(actual_ppsf, pred_ppsf))
mae = mean_absolute_error(actual_ppsf, pred_ppsf)
mape = (np.abs(actual_ppsf - pred_ppsf) / actual_ppsf).mean() * 100

print("=== Corrected OLS Model Performance ===")
print(f"R-squared: {r2:.4f}")
print(f"Adjusted R-squared: {adj_r2:.4f}")
print(f"Smearing Factor: {smearing_factor:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MAPE: {mape:.2f}%")

=== Corrected OLS Model Performance ===
R-squared: 0.3404
Adjusted R-squared: 0.3110
Smearing Factor: 4.2606
RMSE: 389.7961
MAE: 98.4209
MAPE: 4473.24%


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Predictions in log space:
pred_log = model.predict(df_c)
actual_log = df_c["log_ppsf"]

rmse_log = np.sqrt(mean_squared_error(actual_log, pred_log))
mae_log = mean_absolute_error(actual_log, pred_log)

print("RMSE (log space):", rmse_log)
print("MAE (log space):", mae_log)


RMSE (log space): 1.8195768366409277
MAE (log space): 1.4095801722831025


In [ ]:
pred_ppsf_corrected = np.exp(pred_log) * smearing_factor
actual_ppsf = df_c["price_per_sqft"]

ape = np.abs((pred_ppsf_corrected - actual_ppsf) / actual_ppsf) * 100
mdape = np.median(ape)

print("MdAPE:", mdape)


MdAPE: 278.75687432869955


In [ ]:
df2 = eda_df.copy()

# date filtering
df2 = df2[(df2["SALEYEAR"] >= 2010) & (df2["SALEYEAR"] <= 2025)]

# valid prices
df2 = df2[(df2["SALEPRIC"] >= 1000) & (df2["SALEPRIC"] <= 5_000_000)]

# area filtering
df2 = df2[df2["SHAPE_Area"] >= 500]

# ppsf filtering
df2 = df2[(df2["price_per_sqft"] >= 1) & (df2["price_per_sqft"] <= 500)]

print("Remaining rows:", len(df2))
print(df2["price_per_sqft"].describe(percentiles=[0.99, 0.999]))


Remaining rows: 1613
count    1613.000000
mean       51.914185
std        80.175255
min         1.000000
50%        20.000000
99%       408.550000
99.9%     477.466240
max       488.410000
Name: price_per_sqft, dtype: float64


rebuild log variables

In [ ]:
df2["log_ppsf"] = np.log(df2["price_per_sqft"])
df2["log_area"] = np.log(df2["SHAPE_Area"])
df2["log_area_sq"] = df2["log_area"] ** 2

Rebuild categorical variables and neighbour grouping

In [ ]:
df2["quarter"] = df2["SALEDATE"].dt.quarter.astype("category")
df2["year"] = df2["SALEDATE"].dt.year.astype("category")
counts = df2["NEIGHBOR"].value_counts()
small = counts[counts < 10].index
df2["NEIGHBOR_GRP"] = df2["NEIGHBOR"].replace(small, "OTHER").astype("category")


/tmp/ipython-input-2163009008.py:5: FutureWarning:

The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.



In [ ]:
formula2 = """
log_ppsf ~
    log_area +
    log_area_sq +
    C(WARD) +
    C(NEIGHBOR_GRP) +
    C(year) +
    C(quarter)
"""

model2 = smf.ols(formula=formula2, data=df2).fit()


In [ ]:
# predictions in log space
pred_log = model2.predict(df2)

# residuals
resid = df2["log_ppsf"] - pred_log

# duan smearing
smear = np.exp(resid).mean()

# corrected predictions in price space
pred_ppsf = np.exp(pred_log) * smear

# actual
actual_ppsf = df2["price_per_sqft"]

# metrics (correct)
rmse_log = np.sqrt(mean_squared_error(df2["log_ppsf"], pred_log))
mae_log = mean_absolute_error(df2["log_ppsf"], pred_log)

ape = np.abs((pred_ppsf - actual_ppsf) / actual_ppsf) * 100
mdape = np.median(ape)

print("R-squared:", model2.rsquared)
print("Adjusted R2:", model2.rsquared_adj)
print("RMSE (log):", rmse_log)
print("MAE (log):", mae_log)
print("MdAPE:", mdape)
print("Smearing factor:", smear)


R-squared: 0.27764992070703465
Adjusted R2: 0.24190864074201812
RMSE (log): 1.3124748122618288
MAE (log): 1.0641079630628518
MdAPE: 99.08268801122001
Smearing factor: 2.2708819084865706


Vacant land prices in Baltimore exhibit extremely high variability due to micro-location factors, zoning differences, and lot irregularities. Without spatial comparables (distance-weighted recent sales), the hedonic OLS model is expected to achieve low predictive accuracy.

Our OLS model serves as a transparent, interpretable baseline but is not intended to provide highly accurate predictions. Instead, it establishes the fundamental relationships between lot area, neighborhood effects, and time effects. Predictive performance is improved in subsequent gradient boosting models (XGBoost), which better capture nonlinear interactions.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# Fitted values and residuals
fitted = model2.fittedvalues
residuals = model2.resid

res_df = pd.DataFrame({
    "Fitted": fitted,
    "Residuals": residuals
})

fig_res = px.scatter(
    res_df,
    x="Fitted",
    y="Residuals",
    opacity=0.5,
    title="Residuals vs Fitted (OLS)",
    labels={"Fitted": "Fitted Values (log price_per_sqft)",
            "Residuals": "Residuals"}
)

fig_res.add_hline(y=0, line_dash="dash", line_color="red")
fig_res.show()


In [ ]:
# Get coefficients (excluding intercept)
coef_series = model2.params.drop("Intercept", errors="ignore")
coef_df = pd.DataFrame({
    "variable": coef_series.index,
    "coef": coef_series.values
})

# Focus on main effects & key categories (optional filter)
mask = coef_df["variable"].str.contains(
    "log_area|year|quarter|WARD|NEIGHBOR_GRP",
    regex=True
)
coef_sub = coef_df[mask].copy()

# Sort for nicer display
coef_sub = coef_sub.sort_values("coef")

fig_coef = px.bar(
    coef_sub,
    x="coef",
    y="variable",
    orientation="h",
    title="OLS Coefficient Plot",
    labels={"coef": "Coefficient", "variable": "Variable"}
)

fig_coef.add_vline(x=0, line_dash="dash", line_color="red")
fig_coef.update_layout(yaxis=dict(tickfont=dict(size=10)))
fig_coef.show()


In [ ]:
# Predicted log price_per_sqft
pred_log = model2.predict(df2)
actual_log = df2["log_ppsf"]

scatter_df = pd.DataFrame({
    "Actual_log_ppsf": actual_log,
    "Predicted_log_ppsf": pred_log
})

fig_avp = px.scatter(
    scatter_df,
    x="Actual_log_ppsf",
    y="Predicted_log_ppsf",
    opacity=0.5,
    title="Actual vs Predicted (log price_per_sqft)",
    labels={
        "Actual_log_ppsf": "Actual log(price_per_sqft)",
        "Predicted_log_ppsf": "Predicted log(price_per_sqft)"
    }
)

# 45-degree line
min_val = min(scatter_df.min())
max_val = max(scatter_df.max())

fig_avp.add_trace(
    go.Scatter(
        x=[min_val, max_val],
        y=[min_val, max_val],
        mode="lines",
        line=dict(dash="dash", color="red"),
        name="45° line"
    )
)

fig_avp.show()


XGBoost

In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# ---------------------------------------
# 1. Start from your cleaned df2
# ---------------------------------------
data = df2.copy().reset_index(drop=True)

# Make sure SALEDATE is datetime
data["SALEDATE"] = pd.to_datetime(data["SALEDATE"])

# Feature engineering
data["log_ppsf"] = np.log(data["price_per_sqft"])
data["log_area"] = np.log(data["SHAPE_Area"])
data["log_area_sq"] = data["log_area"] ** 2
data["year"] = data["SALEDATE"].dt.year.astype("category")
data["quarter"] = data["SALEDATE"].dt.quarter.astype("category")

# NEIGHBOR_GRP
if "NEIGHBOR_GRP" not in data.columns:
    counts = data["NEIGHBOR"].value_counts()
    small = counts[counts < 10].index
    data["NEIGHBOR_GRP"] = data["NEIGHBOR"].replace(small, "OTHER").astype("category")

# ---------------------------------------
# 2. Define features and target
# ---------------------------------------
num_cols = ["log_area", "log_area_sq"]
cat_cols = ["WARD", "NEIGHBOR_GRP", "year", "quarter"]

for c in cat_cols:
    data[c] = data[c].astype("category")

X = pd.get_dummies(data[num_cols + cat_cols], drop_first=True)
y = data["log_ppsf"]

print("Total rows:", len(data))
print("X shape   :", X.shape)

# ---------------------------------------
# 3. Random 80/20 train-test split
# ---------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print("Train rows:", X_train.shape[0])
print("Test rows :", X_test.shape[0])

# ---------------------------------------
# 4. Train XGBoost
# ---------------------------------------
xgb_model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42
)

xgb_model.fit(X_train, y_train)

# ---------------------------------------
# 5. Evaluation helper
# ---------------------------------------
def eval_model(y_true, y_pred, label):
    rmse_log = np.sqrt(mean_squared_error(y_true, y_pred))
    mae_log = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    pred_ppsf = np.exp(y_pred)
    actual_ppsf = np.exp(y_true)

    ape = np.abs((pred_ppsf - actual_ppsf) / actual_ppsf) * 100
    mdape = np.median(ape)

    print(f"\n=== {label} ===")
    print(f"R² (log space): {r2:.4f}")
    print(f"RMSE (log):     {rmse_log:.4f}")
    print(f"MAE  (log):     {mae_log:.4f}")
    print(f"MdAPE (%):      {mdape:.2f}%")

# ---------------------------------------
# 6. Train & Test metrics
# ---------------------------------------
train_pred = xgb_model.predict(X_train)
test_pred  = xgb_model.predict(X_test)

eval_model(y_train, train_pred, "XGBoost - TRAIN")
eval_model(y_test,  test_pred,  "XGBoost - TEST")


Total rows: 1613
X shape   : (1613, 77)
Train rows: 1290
Test rows : 323

=== XGBoost - TRAIN ===
R² (log space): 0.5603
RMSE (log):     1.0276
MAE  (log):     0.8066
MdAPE (%):      57.11%

=== XGBoost - TEST ===
R² (log space): 0.2749
RMSE (log):     1.2957
MAE  (log):     1.0362
MdAPE (%):      69.74%


In [ ]:
data.columns

Index(['PIN', 'BLOCKLOT', 'SALEDATE', 'SALEDATE_PARSED', 'SALEYEAR',
       'SALEMONTH', 'SALEQTR', 'SALEPRIC', 'SHAPE_Area', 'LOT_ACRES',
       'SHAPE_Length', 'price_per_sqft', 'WARD', 'SECTION', 'NEIGHBOR',
       'join_key_used', 'lon', 'lat', 'log_ppsf', 'log_area', 'year',
       'quarter', 'log_area_sq', 'NEIGHBOR_GRP'],
      dtype='object')

Training XGBoost using Lon & Lat

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from xgboost import XGBRegressor

In [ ]:
# Start from your cleaned data
data = df2.copy().reset_index(drop=True)

# Ensure datetime
data["SALEDATE"] = pd.to_datetime(data["SALEDATE"])

# Basic log features
data["log_ppsf"] = np.log(data["price_per_sqft"])
data["log_area"] = np.log(data["SHAPE_Area"])
data["log_area_sq"] = data["log_area"] ** 2

# Time features
data["year"] = data["SALEDATE"].dt.year.astype("category")
data["quarter"] = data["SALEDATE"].dt.quarter.astype("category")

# NEIGHBOR_GRP (if not already present)
if "NEIGHBOR_GRP" not in data.columns:
    counts = data["NEIGHBOR"].value_counts()
    small = counts[counts < 10].index
    data["NEIGHBOR_GRP"] = data["NEIGHBOR"].replace(small, "OTHER")
    data["NEIGHBOR_GRP"] = data["NEIGHBOR_GRP"].astype("category")

# Make sure lon/lat exist and have no missing values
data = data.dropna(subset=["lon", "lat"])

print("Rows after ensuring lon/lat:", len(data))
print(data[["lon", "lat"]].head())

Rows after ensuring lon/lat: 1613
         lon        lat
0 -76.646824  39.305613
1 -76.646830  39.305747
2 -76.644914  39.303309
3 -76.653408  39.301243
4 -76.639122  39.301777


In [ ]:
# Prepare coordinate matrix
coords = data[["lat", "lon"]].to_numpy()

# --------- A) Nearest Neighbors (5 nearest sales) ----------
# We ask for 6 neighbors so that one of them is the point itself; we'll skip it.
n_neighbors = 6
nbrs = NearestNeighbors(n_neighbors=n_neighbors, metric="euclidean")
nbrs.fit(coords)
distances, indices = nbrs.kneighbors(coords)

knn_mean_ppsf = []
knn_median_ppsf = []
spatial_lag_ppsf = []

for dist_row, idx_row in zip(distances, indices):
    # Skip index 0 (the parcel itself)
    neighbor_idxs = idx_row[1:n_neighbors]
    neighbor_dists = dist_row[1:n_neighbors]

    neighbor_prices = data.iloc[neighbor_idxs]["price_per_sqft"].values

    # Mean and median PPSF of nearest neighbors
    knn_mean_ppsf.append(neighbor_prices.mean())
    knn_median_ppsf.append(np.median(neighbor_prices))

    # Spatial lag: distance-weighted average PPSF
    weights = 1 / (neighbor_dists + 1e-6)  # avoid division by zero
    spatial_lag_ppsf.append(np.average(neighbor_prices, weights=weights))

data["knn5_mean_ppsf"] = knn_mean_ppsf
data["knn5_median_ppsf"] = knn_median_ppsf
data["spatial_lag_ppsf"] = spatial_lag_ppsf

# --------- B) Distance to a reference point (e.g., City Center) ----------
# We'll just use Euclidean distance in lat/lon space as a proxy.
# For Baltimore, roughly center:
center_lat, center_lon = 39.2904, -76.6122

data["dist_center_deg"] = np.sqrt(
    (data["lat"] - center_lat) ** 2 + (data["lon"] - center_lon) ** 2
)

# --------- C) Spatial Clusters using KMeans ----------
kmeans = KMeans(n_clusters=20, random_state=42, n_init=10)
data["spatial_cluster"] = kmeans.fit_predict(coords)
data["spatial_cluster"] = data["spatial_cluster"].astype("category")

data[[
    "price_per_sqft", "knn5_mean_ppsf", "spatial_lag_ppsf",
    "dist_center_deg", "spatial_cluster"
]].head()


,price_per_sqft,knn5_mean_ppsf,spatial_lag_ppsf,dist_center_deg,spatial_cluster
0,24.77,12.488,8.005111,0.037819,14
1,7.01,108.028,33.516864,0.037879,14
2,28.32,9.560,10.247765,0.035169,14
3,177.17,46.390,38.422017,0.042611,14
4,1.85,3.976,3.870786,0.029227,14


In [ ]:
# Numeric features
num_cols = [
    "log_area",
    "log_area_sq",
    "knn5_mean_ppsf",
    "knn5_median_ppsf",
    "spatial_lag_ppsf",
    "dist_center_deg"
]

# Categorical features
cat_cols = [
    "WARD",
    "NEIGHBOR_GRP",
    "year",
    "quarter",
    "spatial_cluster"
]

# Ensure correct dtypes
for c in cat_cols:
    data[c] = data[c].astype("category")

# Target
y = data["log_ppsf"]

# One-hot encode categoricals
X = pd.get_dummies(
    data[num_cols + cat_cols],
    drop_first=True
)

print("Total rows:", len(data))
print("X shape   :", X.shape)

# Random 80/20 split (since we don't have reliable 2024+ test data)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print("Train rows:", X_train.shape[0])
print("Test rows :", X_test.shape[0])


Total rows: 1613
X shape   : (1613, 100)
Train rows: 1290
Test rows : 323


In [ ]:
xgb_spatial = XGBRegressor(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42
)

xgb_spatial.fit(X_train, y_train)

print("Model trained with spatial features.")


Model trained with spatial features.


In [ ]:
def eval_model(y_true, y_pred, label):
    rmse_log = np.sqrt(mean_squared_error(y_true, y_pred))
    mae_log = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    pred_ppsf = np.exp(y_pred)
    actual_ppsf = np.exp(y_true)

    ape = np.abs((pred_ppsf - actual_ppsf) / actual_ppsf) * 100
    mdape = np.median(ape)

    print(f"\n=== {label} ===")
    print(f"R² (log space): {r2:.4f}")
    print(f"RMSE (log):     {rmse_log:.4f}")
    print(f"MAE  (log):     {mae_log:.4f}")
    print(f"MdAPE (%):      {mdape:.2f}%")

# Predictions
train_pred_spatial = xgb_spatial.predict(X_train)
test_pred_spatial  = xgb_spatial.predict(X_test)

# Evaluate
eval_model(y_train, train_pred_spatial, "Spatial XGBoost - TRAIN")
eval_model(y_test,  test_pred_spatial,  "Spatial XGBoost - TEST")



=== Spatial XGBoost - TRAIN ===
R² (log space): 0.9408
RMSE (log):     0.3770
MAE  (log):     0.2750
MdAPE (%):      19.24%

=== Spatial XGBoost - TEST ===
R² (log space): 0.4511
RMSE (log):     1.1274
MAE  (log):     0.8374
MdAPE (%):      50.86%


Incorporating spatial features derived from parcel coordinates—namely nearest-neighbor price features, distance-based spatial lags, and coordinate-based micro-clusters—substantially improved the predictive performance of the model.

The non-spatial XGBoost model achieved R² ≈ 0.27 and MdAPE ≈ 70%, whereas the spatial XGBoost model improved test R² to ≈ 0.45 and reduced MdAPE to ≈ 51%. This demonstrates that spatial proximity to recent comparable sales is the strongest determinant of vacant land price, and significantly enhances valuation accuracy.

SHAP Explanation

In [ ]:
!pip install shap --upgrade

In [ ]:
import shap
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Explain the trained spatial model
explainer = shap.TreeExplainer(xgb_spatial)
shap_values = explainer(X_train)

In [ ]:
# Compute mean(|SHAP value|)
importance = np.abs(shap_values.values).mean(axis=0)

importance_df = pd.DataFrame({
    "feature": X_train.columns,
    "importance": importance
}).sort_values("importance", ascending=False)

fig = px.bar(
    importance_df.head(20),  # top 20 features
    x="importance",
    y="feature",
    orientation="h",
    title="Top SHAP Feature Importances (Plotly)",
    height=700
)

fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()


In [ ]:
shap.plots.beeswarm(
    shap_values,
    max_display=20,
    show=False
)

# Convert the Matplotlib figure into a Plotly figure
# NOTE: shap doesn't natively return Plotly, so we use a simple workaround:
import matplotlib.pyplot as plt
import plotly.tools as tls

fig_mpl = plt.gcf()
fig_plotly = tls.mpl_to_plotly(fig_mpl)
fig_plotly.update_layout(title="SHAP Summary (Beeswarm) – Plotly")
fig_plotly.show()


/usr/local/lib/python3.12/dist-packages/plotly/matplotlylib/mplexporter/exporter.py:87: UserWarning:

Blended transforms not yet supported. Zoom behavior may not work as expected.

/usr/local/lib/python3.12/dist-packages/plotly/matplotlylib/renderer.py:507: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

/usr/local/lib/python3.12/dist-packages/plotly/matplotlylib/renderer.py:571: UserWarning:

Dang! That path collection is out of this world. I totally don't know what to do with it yet! Plotly can only import path collections linked to 'data' coordinates



In [ ]:
def plot_shap_dependence(feature_name):
    idx = list(X_train.columns).index(feature_name)
    shap_vals = shap_values.values[:, idx]
    feat_vals = X_train[feature_name]

    fig = px.scatter(
        x=feat_vals,
        y=shap_vals,
        color=feat_vals,
        title=f"SHAP Dependence Plot – {feature_name}",
        labels={"x": feature_name, "y": "SHAP value"},
        color_continuous_scale="Turbo"
    )
    fig.show()


In [ ]:
plot_shap_dependence("log_area")
plot_shap_dependence("knn5_mean_ppsf")
plot_shap_dependence("spatial_lag_ppsf")
plot_shap_dependence("dist_center_deg")


SHAP analysis demonstrates that the model relies primarily on local comparable sales. The strongest feature is spatial_lag_ppsf, representing the average PPSF of nearby recent sales, followed closely by lot size (log_area) and nearest-neighbor statistics (knn5_mean_ppsf).

These findings confirm that vacant land value in Baltimore is highly localized—properties located near expensive comps receive positive SHAP contributions, while parcels in low-priced clusters show negative contributions.

Additionally, distance to city center has a meaningful spatial gradient, reflecting Baltimore’s urban price structure. Overall, the SHAP plots indicate that the model’s reasoning mirrors real-world appraisal logic.

Plotly Visualizations

In [ ]:
import plotly.express as px
import pandas as pd

# Create comparison dataframe
compare_df = pd.DataFrame({
    "Actual_log_ppsf": y_test,
    "Pred_log_ppsf": test_pred
})

fig = px.scatter(
    compare_df,
    x="Actual_log_ppsf",
    y="Pred_log_ppsf",
    title="Actual vs Predicted (Log Price Per Sqft) - Spatial XGBoost",
    opacity=0.7,
    trendline="ols",
    labels={"Actual_log_ppsf": "Actual Log(PPSF)", "Pred_log_ppsf": "Predicted Log(PPSF)"}
)

fig.update_layout(
    width=800,
    height=600
)

fig.show()


The Actual vs Predicted plot shows that predictions increase in line with true log prices, demonstrating good calibration. The spread widens at higher PPSF values due to the inherent volatility in premium land parcels, which is common in real estate datasets.

In [ ]:
import numpy as np

# Residuals
residuals = y_test - test_pred

fig = px.histogram(
    residuals,
    nbins=60,
    title="Residual Distribution (Log Space) - Spatial XGBoost",
    labels={"value": "Residual (Actual - Predicted)", "count": "Frequency"},
    opacity=0.75
)

fig.update_layout(
    width=800,
    height=600
)

fig.show()


Residuals are centered around zero, indicating that the model is unbiased overall. The right-skew in residuals is expected due to extreme high-PPSF outliers typical in vacant land sales.

# **7. Results**

## **7.1 Model Comparison Overview**

Three models were evaluated to predict log(price per square foot) of vacant land in Baltimore:

| Model | Test R² | RMSE (log) | MAE (log) | MdAPE (%) | Notes |
|------|---------|------------|-----------|------------|-------|
| **OLS (Baseline)** | ~0.24 | High | High | ~99% | Limited by linearity and lack of spatial features |
| **XGBoost (Non-Spatial)** | ~0.27 | Lower | Lower | ~70% | Learns nonlinear patterns but still lacks spatial context |
| **Spatial XGBoost (Final Model)** | **0.45** | **1.13** | **0.84** | **~51%** | Major performance gain from spatial feature engineering |

> **Key Finding:**  
> Spatial features almost **doubled the predictive performance**, proving that land valuation is strongly influenced by local comparable sales.

---

## **7.2 Performance of the Final Spatial XGBoost Model**

The Spatial XGBoost model achieved the strongest accuracy:

- **Test R² = 0.451**
- **RMSE_log = 1.127**
- **MAE_log = 0.837**
- **Median Absolute Percentage Error (MdAPE) ≈ 50.86%**

These values are realistic for land price modeling, a domain known for high variability and sparse comparable sales.

---

## **7.3 Actual vs Predicted (Log PPSF)**

**Actual vs Predicted Plot:**  
*(Insert the Plotly chart here)*

### **Interpretation**
- A clear **positive linear trend** indicates strong calibration.
- Most mid-range parcels are predicted accurately.
- Wider dispersion for high-value parcels is expected due to extreme outliers and high volatility.

> **Conclusion:**  
> The model successfully captures Baltimore’s overall land value structure.

---

## **7.4 Residual Analysis**

**Residual Distribution Plot:**  
*(Insert the Plotly chart here)*

### **Interpretation**
- Residuals are centered around **zero**, showing the model is unbiased.
- The right-skew tail indicates **underprediction** for extreme high-PPSF parcels — normal in land valuation.
- No systematic overprediction or underprediction pattern.

> **Conclusion:**  
> Residuals are statistically well-behaved, indicating stable model performance.

---

## **7.5 SHAP Interpretability: What Drives Baltimore Land Value?**

SHAP analysis reveals how individual features influence predictions.

### **Top Features Affecting Price:**
1. **Spatial Lag (PPSF of nearby parcels)**
2. **Lot Size (log_area)**
3. **KNN Comparable Prices (mean & median)**
4. **Distance to City Center**
5. **Spatial Clusters (KMeans)**
6. **Neighborhood Groups**

### **Key Insights**

#### **A. Local Comparable Sales Are the Strongest Driver**
Parcels near recent high-priced sales receive **positive SHAP values**.  
Parcels near lower-price comps receive **negative SHAP values**.

➡ This matches real appraisal practice.

#### **B. Lot Size Has a Clear PPSF Effect**
- Small lots → **higher price/ft²**
- Large lots → **lower price/ft²**

#### **C. Distance to City Center Matters**
Closer parcels tend to have **higher land value**.

#### **D. Spatial Micro-Clusters Capture Local Effects**
Spatial clusters help the model detect patterns **not captured by official neighborhoods**.

---

## **7.6 Summary of Findings**

- The **OLS baseline** performed poorly due to its simplicity.
- **Non-spatial XGBoost** improved performance but lacked geographic awareness.
- **Spatial XGBoost showed the largest performance jump**, highlighting:
  - The importance of nearest comparable sales  
  - The role of lot size  
  - The strong spatial structure of Baltimore's land market  

> **Final Conclusion:**  
> The Spatial XGBoost model provides a robust, explainable, and realistic estimate of vacant land prices in Baltimore. Performance metrics and SHAP insights align closely with professional appraisal theory.

---



In [ ]:
OUT_FILE = f"{out_dir}/final_vacant_land_eda.csv"
eda_df.to_csv(OUT_FILE, index=False)
print(f"EDA dataframe saved to: {OUT_FILE}")

EDA dataframe saved to: /content/drive/MyDrive/Capstone/processed/final_vacant_land_eda.csv


App building

In [ ]:
# === Cell A: Set up app folder and copy data ===
import os
import shutil

CAPSTONE_DIR = "/content/drive/MyDrive/Capstone"
PROJECT_DIR = os.path.join(CAPSTONE_DIR, "vacant_land_app")

os.makedirs(PROJECT_DIR, exist_ok=True)

# NOTE: final CSV is in the 'processed' folder
SRC = os.path.join(CAPSTONE_DIR, "processed", "final_vacant_land_eda.csv")
DST = os.path.join(PROJECT_DIR, "final_vacant_land_eda.csv")

shutil.copy(SRC, DST)

os.chdir(PROJECT_DIR)
print("Current working directory:", os.getcwd())
print("Project contents:", os.listdir())


Current working directory: /content/drive/MyDrive/Capstone/vacant_land_app
Project contents: ['cloudflared', 'final_vacant_land_eda.csv', 'streamlit_app.py', 'streamlit.log']


In [ ]:
# === Cell B: Install dependencies (Colab-only, NOT needed in Streamlit Cloud) ===
!pip install -q streamlit xgboost scikit-learn pandas numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 76.0 MB/s eta 0:00:00


In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd
import numpy as np

from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from xgboost import XGBRegressor

# -----------------------------
# 1. Data loading & preparation
# -----------------------------
@st.cache_data
def load_data():
    # CSV must be in the same folder as this script
    df = pd.read_csv("final_vacant_land_eda.csv")

    # Basic cleaning / sanity checks
    df = df.dropna(subset=["price_per_sqft", "SHAPE_Area", "lon", "lat"])
    df = df[(df["price_per_sqft"] > 0) & (df["SHAPE_Area"] > 0)]

    # Parse SALEDATE if needed
    df["SALEDATE"] = pd.to_datetime(df["SALEDATE"])

    # Core log features
    df["log_ppsf"] = np.log(df["price_per_sqft"])
    df["log_area"] = np.log(df["SHAPE_Area"])
    df["log_area_sq"] = df["log_area"] ** 2

    # Time features
    df["year"] = df["SALEDATE"].dt.year.astype("category")
    df["quarter"] = df["SALEDATE"].dt.quarter.astype("category")

    # Neighborhood grouping
    if "NEIGHBOR_GRP" not in df.columns:
        counts = df["NEIGHBOR"].value_counts()
        small = counts[counts < 10].index
        df["NEIGHBOR_GRP"] = df["NEIGHBOR"].replace(small, "OTHER")
    df["NEIGHBOR_GRP"] = df["NEIGHBOR_GRP"].astype("category")

    return df


@st.cache_data
def add_spatial_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add KNN-based comps, spatial lag, distance to center, and clusters."""
    df = df.copy()
    coords = df[["lat", "lon"]].to_numpy()

    # --- KNN features (5 nearest neighbors) ---
    n_neighbors = 6  # self + 5 neighbors
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, metric="euclidean")
    nbrs.fit(coords)
    distances, indices = nbrs.kneighbors(coords)

    knn_mean_ppsf = []
    knn_median_ppsf = []
    spatial_lag_ppsf = []

    for dist_row, idx_row in zip(distances, indices):
        neighbor_idxs = idx_row[1:n_neighbors]
        neighbor_dists = dist_row[1:n_neighbors]
        neighbor_prices = df.iloc[neighbor_idxs]["price_per_sqft"].values

        knn_mean_ppsf.append(neighbor_prices.mean())
        knn_median_ppsf.append(np.median(neighbor_prices))

        weights = 1 / (neighbor_dists + 1e-6)
        spatial_lag_ppsf.append(np.average(neighbor_prices, weights=weights))

    df["knn5_mean_ppsf"] = knn_mean_ppsf
    df["knn5_median_ppsf"] = knn_median_ppsf
    df["spatial_lag_ppsf"] = spatial_lag_ppsf

    # --- Rough distance to Baltimore center (deg) ---
    center_lat, center_lon = 39.2904, -76.6122
    df["dist_center_deg"] = np.sqrt(
        (df["lat"] - center_lat) ** 2 + (df["lon"] - center_lon) ** 2
    )

    # --- Spatial clusters (numeric here; we one-hot later) ---
    kmeans = KMeans(n_clusters=20, random_state=42, n_init=10)
    df["spatial_cluster"] = kmeans.fit_predict(coords)

    return df


@st.cache_resource
def train_spatial_xgb(df: pd.DataFrame):
    """Train a spatial XGBoost model on log(price_per_sqft)."""
    df = df.copy()

    num_cols = [
        "log_area",
        "log_area_sq",
        "knn5_mean_ppsf",
        "knn5_median_ppsf",
        "spatial_lag_ppsf",
        "dist_center_deg",
    ]

    cat_cols = ["WARD", "NEIGHBOR_GRP", "year", "quarter", "spatial_cluster"]

    for c in cat_cols:
        df[c] = df[c].astype("category")

    y = df["log_ppsf"]
    X = pd.get_dummies(df[num_cols + cat_cols], drop_first=True)

    model = XGBRegressor(
        n_estimators=600,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42,
    )

    model.fit(X, y)

    return model, X, y, num_cols, cat_cols


# -----------------------------
# 2. Streamlit UI
# -----------------------------
st.set_page_config(page_title="Baltimore Vacant Land Valuation", layout="wide")

st.title("🧮 Spatial Vacant Land Valuation – Baltimore")
st.write(
    "This app uses a spatial XGBoost model trained on vacant land sales "
    "to estimate **price per square foot** and **total land value** for a selected parcel."
)

with st.spinner("Loading and preparing data..."):
    df = load_data()
    df = add_spatial_features(df)
    model, X_all, y_all, num_cols, cat_cols = train_spatial_xgb(df)

st.success(f"Data loaded. Parcels available: {len(df)}")

# Sidebar: filters and PIN selection
st.sidebar.header("🔎 Parcel Selection")

neigh_options = ["All"] + sorted(df["NEIGHBOR"].dropna().unique().tolist())
selected_neigh = st.sidebar.selectbox("Filter by NEIGHBOR:", neigh_options)

if selected_neigh != "All":
    df_sub = df[df["NEIGHBOR"] == selected_neigh].copy()
else:
    df_sub = df.copy()

pin_options = df_sub["PIN"].astype(str).unique().tolist()
selected_pin = st.sidebar.selectbox("Select Parcel PIN:", sorted(pin_options))

# Get row and corresponding X row
row = df[df["PIN"].astype(str) == selected_pin].iloc[0]
idx = row.name
X_row = X_all.loc[[idx]]

# Prediction in log space
pred_log_ppsf = model.predict(X_row)[0]
pred_ppsf = float(np.exp(pred_log_ppsf))
pred_total_price = pred_ppsf * float(row["SHAPE_Area"])

# -----------------------------
# 3. Display details & results
# -----------------------------
st.subheader("📍 Selected Parcel Details")

col1, col2, col3 = st.columns(3)

with col1:
    st.metric("PIN", selected_pin)
    st.write(f"**WARD:** {row['WARD']}")
    st.write(f"**SECTION:** {row['SECTION']}")

with col2:
    st.write(f"**NEIGHBOR:** {row['NEIGHBOR']}")
    st.write(f"**Lot Area (sqft):** {row['SHAPE_Area']:.0f}")
    st.write(f"**Lat, Lon:** {row['lat']:.5f}, {row['lon']:.5f}")

with col3:
    if not pd.isna(row.get("SALEPRIC", np.nan)):
        st.write(f"**Recorded SALEPRICE:** ${row['SALEPRIC']:,.0f}")
        st.write(f"**Recorded price/ft²:** ${row['price_per_sqft']:.2f}")
    else:
        st.write("No recorded sale price available.")

st.subheader("📊 Model Valuation (Spatial XGBoost)")

col4, col5 = st.columns(2)

with col4:
    st.metric("Predicted price per ft²", f"${pred_ppsf:,.2f}")
    st.metric("Predicted land value", f"${pred_total_price:,.0f}")

with col5:
    if not pd.isna(row.get("SALEPRIC", np.nan)):
        actual_ppsf = row["price_per_sqft"]
        ape = abs(pred_ppsf - actual_ppsf) / actual_ppsf * 100
        st.write(f"**Actual price/ft²:** ${actual_ppsf:,.2f}")
        st.write(f"**Absolute % error vs actual:** {ape:,.1f}%")
    else:
        st.write("No actual sale price to compare for this parcel.")

st.markdown("---")
st.caption(
    "Model: Spatial XGBoost on log(price_per_sqft) using lot size, time, "
    "nearest-neighbor PPSF, spatial lag, distance to center, and spatial clusters. "
    "Developed for UMBC DATA606 Capstone."
)


Overwriting streamlit_app.py


In [ ]:
# === Cell D: Run Streamlit (Colab preview only) ===
!pkill -f streamlit || true
!fuser -n tcp -k 8501 || true

!streamlit run streamlit_app.py --server.port 8501 --server.address 0.0.0.0 &> streamlit.log &

!sleep 5
!tail -n 40 streamlit.log


^C




In [ ]:
# === Cell E: Start Cloudflare tunnel (ONLY if you want a public URL) ===
!wget -q -O cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared

!./cloudflared tunnel --url http://localhost:8501 --no-autoupdate


2025-11-21T00:58:12Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-21T00:58:12Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-21T00:58:15Z INF +--------------------------------------------------------------------------------------------+
2025-11-21T00:58:15Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-21T00:58:15Z INF |  https://bali-newman-ntsc-automobile.trycloudflare.com